# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [2]:
import os

API_KEY = os.environ["ANTHROPIC_API_KEY"]
MODEL_NAME = "claude-3-haiku-20240307"

In [3]:
%pip install anthropic --quiet

# Import python's built-in regular expression library
import re
import anthropic

client = anthropic.Anthropic(api_key=API_KEY)


# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages,
    )
    return message.content[0].text


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Claude to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [4]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [{"role": "user", "content": first_user}]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here are ten words that all end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Jab
5. Lab
6. Nab
7. Slab
8. Tab
9. Wab
10. Zab


**Asking Claude to make its answer more accurate** fixes the error! 

Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [5]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are ten words that all end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Jab\n5. Lab\n6. Nab\n7. Slab\n8. Tab\n9. Wab\n10. Zab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are ten real words that all end with the exact letters 'ab':

1. Cab
2. Dab
3. Grab
4. Jab
5. Lab
6. Nab
7. Slab
8. Tab
9. Crab
10. Blab


But is Claude revising its answer just because we told it to? What if we start off with a correct answer already? Will Claude lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [6]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Crab


You may notice that if you generate a respnse from the above block a few times, Claude leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Claude an out! Let's try this one more time.

In [7]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]

------------------------------------- Claude's response -------------------------------------
Okay, here is the list with replacements for any words that are not real:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

All of these are real words, so the original list is returned.


Try generating responses from the above code a few times to see that Claude is much better at sticking to its guns now.

You can also use prompt chaining to **ask Claude to make its responses better**. Below, we asked Claude to first write a story, and then improve the story it wrote. Your personal tastes may vary, but many might agree that Claude's second version is better.

First, let's generate Claude's first version of the story.

In [8]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [{"role": "user", "content": first_user}]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here is a three-sentence short story about a girl who likes to run:

Amelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that nothing else could match. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it.


Now let's have Claude improve on its first draft.

In [9]:
second_user = "Make the story better."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Write a three-sentence short story about a girl who likes to run.'}, {'role': 'assistant', 'content': 'Here is a three-sentence short story about a girl who likes to run:\n\nAmelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that nothing else could match. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it.'}, {'role': 'user', 'content': 'Make the story better.'}]

------------------------------------- Claude's response -------------------------------------
Here is the three-sentence short story with some improvements:

Amelia's heart raced as she pushed herself harder, her legs pumping with each stride as she flew down

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, Claude's former responses, and so on. You can also **use substitution to do what we call "function calling," which is asking Claude to perform some function, and then taking the results of that function and asking Claude to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is one more example of taking the results of one call to Claude and plugging it into another, longer call. Let's start with the first prompt (which includes prefilling Claude's response this time).

In [10]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": prefill},
]

# Store and print Claude's response
first_response = get_completion(messages)
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(first_response)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>'}]

------------------------------------- Claude's response -------------------------------------

Jesse
Erin
Joey
Keisha
Mel
</names>


Let's pass this list of names into another prompt.

In [11]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": prefill + "\n" + first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>\n\nJesse\nErin\nJoey\nKeisha\nMel\n</names>'}, {'role': 'user', 'content': 'Alphabetize the list.'}]

------------------------------------- Claude's response -------------------------------------
<names>
Erin
Jesse
Joey
Keisha
Mel
</names>


Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [12]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [{"role": "user", "content": first_user}]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here are ten words that all end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Jab
5. Lab
6. Nab
7. Slab
8. Tab
9. Wab
10. Zab


In [13]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are ten words that all end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Jab\n5. Lab\n6. Nab\n7. Slab\n8. Tab\n9. Wab\n10. Zab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are ten real words that all end with the exact letters 'ab':

1. Cab
2. Dab
3. Grab
4. Jab
5. Lab
6. Nab
7. Slab
8. Tab
9. Crab
10. Blab


In [14]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Crab


In [15]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]

------------------------------------- Claude's response -------------------------------------
Okay, here is the list with replacements for any words that are not real:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

All of these are real words, so the original list is returned.


In [16]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [{"role": "user", "content": first_user}]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here is a three-sentence short story about a girl who likes to run:

Amelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that nothing else could match. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it.


In [17]:
second_user = "Make the story better."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Write a three-sentence short story about a girl who likes to run.'}, {'role': 'assistant', 'content': 'Here is a three-sentence short story about a girl who likes to run:\n\nAmelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that nothing else could match. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it.'}, {'role': 'user', 'content': 'Make the story better.'}]

------------------------------------- Claude's response -------------------------------------
Here is the three-sentence short story with some improvements:

Amelia's heart raced as she pushed herself harder, her legs pumping with each stride as she flew down

In [18]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": prefill},
]

# Store and print Claude's response
first_response = get_completion(messages)
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(first_response)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>'}]

------------------------------------- Claude's response -------------------------------------

Jesse
Erin
Joey
Keisha
Mel
</names>


In [19]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {"role": "user", "content": first_user},
    {"role": "assistant", "content": prefill + "\n" + first_response},
    {"role": "user", "content": second_user},
]

# Print Claude's response
print(
    "------------------------ Full messsages array with variable substutions ------------------------"
)
print(messages)
print(
    "\n------------------------------------- Claude's response -------------------------------------"
)
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>\n\nJesse\nErin\nJoey\nKeisha\nMel\n</names>'}, {'role': 'user', 'content': 'Alphabetize the list.'}]

------------------------------------- Claude's response -------------------------------------
<names>
Erin
Jesse
Joey
Keisha
Mel
</names>
